In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GitHub/ERA-V1/Assignments/S9

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/ERA-V1/Assignments/S9


In [2]:
%%writefile src/data_setup.py
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader


NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir, test_dir, train_transforms, test_transforms, batch_size, num_workers=NUM_WORKERS):

  # Download the dataset
  train_data = datasets.CIFAR10(train_dir, train=True, download=True, transform=train_transforms)
  test_data = datasets.CIFAR10(test_dir, train=False, download=True, transform=test_transforms)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Overwriting src/data_setup.py


In [3]:
%%writefile src/model_builder.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, bias=False),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, bias=False),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, bias=False),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 10, kernel_size=1, stride=1, bias=False))
        self.gap = nn.Sequential(
            nn.AdaptiveAvgPool2d(1))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.gap(x)
        x = x.view((x.shape[0],-1))
        x = F.log_softmax(x, dim=1)

        return x


Overwriting src/model_builder.py


In [4]:
%%writefile src/engine.py
from src.utils import plot_graph, show_incorrect_images
import torch

from tqdm.auto import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train_step(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc = 100*correct/processed
  return train_loss, train_acc

def test_step(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0
    test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss

            pred = output.argmax(dim=1)
            correct_mask = pred.eq(target)
            incorrect_indices = ~correct_mask

            test_incorrect_pred['images'].extend(data[incorrect_indices])
            test_incorrect_pred['ground_truths'].extend(target[incorrect_indices])
            test_incorrect_pred['predicted_vals'].extend(pred[incorrect_indices])

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)


    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, test_acc, test_incorrect_pred

def train(model, train_loader, test_loader, device, optimizer, epochs, criterion):

    class_map = {
        0: 'airplane',
        1: 'automobile',
        2: 'bird',
        3: 'cat',
        4: 'deer',
        5: 'dog',
        6: 'frog',
        7: 'horse',
        8: 'ship',
        9: 'truck'
    }
    # Data to plot accuracy and loss graphs
    # Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }


    for epoch in range(epochs):
        print(f'Epoch {epoch}')
        train_loss, train_acc = train_step(model=model, device=device, train_loader=train_loader, optimizer=optimizer, criterion=criterion)
        test_loss, test_acc, test_incorrect_pred = test_step(model=model, device=device, test_loader=test_loader, criterion=criterion)

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    plot_graph(results["train_loss"], results["test_loss"], results["train_acc"], results["test_acc"])
    show_incorrect_images(test_incorrect_pred, class_map)

    # Return the filled results at the end of the epochs
    return results

Overwriting src/engine.py


In [5]:
%%writefile src/utils.py
import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def save_model(model, target_dir, model_name):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

def plot_graph(train_losses, test_losses, train_acc, test_acc):
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Plot Train and Test Loss
    axs[0].plot(train_losses, label='Train Loss')
    axs[0].plot(test_losses, label='Test Loss')
    axs[0].set_title("Train and Test Loss")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].legend()

    # Plot Train and Test Accuracy
    axs[1].plot(train_acc, label='Train Accuracy')
    axs[1].plot(test_acc, label='Test Accuracy')
    axs[1].set_title("Train and Test Accuracy")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].legend()

    plt.savefig("models/loss_accuracy_plot.png")

def show_incorrect_images(test_incorrect_pred, class_map):
    num_images = 10
    num_rows = 2
    num_cols = (num_images + 1) // 2  # Adjust the number of columns based on the number of images

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols * 2, num_rows * 2))

    for i in range(num_images):
        row_idx = i // num_cols
        col_idx = i % num_cols

        img = test_incorrect_pred['images'][i].cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Normalize the image data
        label = test_incorrect_pred['ground_truths'][i].cpu().item()
        pred = test_incorrect_pred['predicted_vals'][i].cpu().item()

        axs[row_idx, col_idx].imshow(img)
        axs[row_idx, col_idx].set_title(f'GT: {class_map[label]}, Pred: {class_map[pred]}')
        axs[row_idx, col_idx].axis('off')

    plt.savefig("models/incorrect_images.png")

Overwriting src/utils.py


In [6]:
%%writefile train.py
import os
import torch
from src import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 2
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
MOMENTUM = 0.9

# Setup directories
train_dir = "../data"
test_dir = "../data"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
                                       ])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.Model1().to(device)

# Set loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)


# Start training with help from engine.py
engine.train(model=model,
             train_loader=train_dataloader,
             test_loader=test_dataloader,
             criterion=criterion,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting train.py


In [7]:
!python train.py

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
Train: Loss=1.8658 Batch_id=1562 Accuracy=14.23: 100% 1563/1563 [00:24<00:00, 63.29it/s]
Test set: Average loss: 0.0637, Accuracy: 2407/10000 (24.07%)

Epoch 1
Train: Loss=1.5459 Batch_id=1562 Accuracy=30.02: 100% 1563/1563 [00:20<00:00, 76.01it/s]
Test set: Average loss: 0.0545, Accuracy: 3605/10000 (36.05%)

[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


In [ ]:
!git status

Refresh index: 100% (27/27), done.


In [ ]:
!